In [ ]:
cd /content/drive/MyDrive/Precog_Task/

/content/drive/MyDrive/Precog_Task


In [ ]:
!git clone https://github.com/uclanlp/corefBias.git

Cloning into 'corefBias'...
remote: Enumerating objects: 543, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 543 (delta 16), reused 7 (delta 2), pack-reused 506 (from 1)
Receiving objects: 100% (543/543), 84.24 MiB | 13.77 MiB/s, done.
Resolving deltas: 100% (314/314), done.
Updating files: 100% (89/89), done.


In [ ]:
cd corefBias/WinoBias/wino/data/

/content/drive/MyDrive/Precog_Task/corefBias/WinoBias/wino/data


In [ ]:
ls

anti_stereotyped_type1.txt.dev   male_occupations.txt
anti_stereotyped_type1.txt.test  pro_stereotyped_type1.txt.dev
anti_stereotyped_type2.txt.dev   pro_stereotyped_type1.txt.test
anti_stereotyped_type2.txt.test  pro_stereotyped_type2.txt.dev
conll_format/                    pro_stereotyped_type2.txt.test
female_occupations.txt           Readme.md


In [ ]:
cd ..

/content/drive/MyDrive/Precog_Task/corefBias/WinoBias/wino


In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
import torch
import re

model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForMaskedLM.from_pretrained(model_name)
model.eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

In [ ]:
import re

def parse_and_mask(sentence, tokenizer):

    match = re.search(r"\b(he|she|him|her|He|She|Him|Her)\b", sentence)
    if not match:
        return None, None

    gold = match.group(0).lower()
    masked = re.sub(r"\b(he|she|him|her|He|She|Him|Her)\b", tokenizer.mask_token, sentence)
    return masked, gold

In [ ]:
def load_sentences(file_path, tokenizer):
    masked_sents, gold_pronouns = [], []
    with open(file_path, "r") as f:
        for line in f:
            sent = line.strip()
            if not sent:
                continue
            masked, gold = parse_and_mask(sent, tokenizer)
            if masked and gold:
                masked_sents.append(masked)
                gold_pronouns.append(gold)
    return masked_sents, gold_pronouns


pro_file = "data/pro_stereotyped_type1.txt.test"
anti_file = "data/anti_stereotyped_type1.txt.test"

pro_sents, pro_labels = load_sentences(pro_file, tokenizer)
anti_sents, anti_labels = load_sentences(anti_file, tokenizer)


print("Pro example:", pro_sents[0])
print("Anti example:", anti_sents[0])

Pro example: 1 The janitor reprimanded [the accountant] because [[MASK]] made a mistake filing paperwork.
Anti example: 1 The janitor reprimanded [the accountant] because [[MASK]] made a mistake filing paperwork.


In [ ]:
def load_occupations(file_path):
    with open(file_path, "r") as f:
        return [line.strip().lower() for line in f if line.strip()]

male_occs = load_occupations("data/male_occupations.txt")
female_occs = load_occupations("data/female_occupations.txt")

print("Male occupations:", male_occs[:5])
print("Female occupations:", female_occs[:5])


Male occupations: ['driver', 'supervisor', 'janitor', 'cook', 'mover']
Female occupations: ['attendant', 'cashier', 'teacher', 'nurse', 'assistant']


In [ ]:
def get_pronoun_probs(sentence, target_pronouns):
    inputs = tokenizer(sentence, return_tensors="pt")

    # [MASK]
    mask_positions = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]
    if len(mask_positions) == 0:
        return None

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    mask_logits = logits[0, mask_positions[0], :]
    probs = torch.softmax(mask_logits, dim=-1)

    results = {}
    for pronoun in target_pronouns:
        token_id = tokenizer.convert_tokens_to_ids(pronoun)
        results[pronoun] = probs[token_id].item()
    return results


In [ ]:
def evaluate_bias(sentences, gold_pronouns, tokenizer, model):
    correct, total = 0, 0
    male_pref, female_pref = 0, 0

    for sent, gold in zip(sentences, gold_pronouns):
        probs = get_pronoun_probs(sent,  ["him", "her", "he", "she"])
        if not probs:
            continue

        predicted = max(probs, key=probs.get)
        if predicted == gold:
            correct += 1

        if predicted in ["he", "him"]:
            male_pref += 1
        elif predicted in ["she", "her"]:
            female_pref += 1

        total += 1

    acc = correct / total if total > 0 else 0
    return acc, male_pref, female_pref, total

In [ ]:
pro_acc, pro_male, pro_female, pro_total = evaluate_bias(pro_sents, pro_labels, tokenizer, model)
anti_acc, anti_male, anti_female, anti_total = evaluate_bias(anti_sents, anti_labels, tokenizer, model)

print("Pro-stereotyped accuracy:", pro_acc)
print("Anti-stereotyped accuracy:", anti_acc)
print("Gap:", pro_acc - anti_acc)

print("Pro male vs female predictions:", pro_male, pro_female)
print("Anti male vs female predictions:", anti_male, anti_female)

Pro-stereotyped accuracy: 0.5597826086956522
Anti-stereotyped accuracy: 0.52
Gap: 0.0397826086956522
Pro male vs female predictions: 296 72
Anti male vs female predictions: 297 78


In [ ]:
import numpy as np

def get_pronoun_probs(sentence, tokenizer, model, pronouns=["he", "she", "him", "her"]):
    """Return probability distribution over candidate pronouns."""
    inputs = tokenizer(sentence, return_tensors="pt")
    mask_indices = (inputs["input_ids"] == tokenizer.mask_token_id).nonzero(as_tuple=True)[1]


    mask_idx = mask_indices[-1].item()

    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits[0, mask_idx]
    probs = torch.nn.functional.softmax(logits, dim=-1)
    return {p: probs[tokenizer.convert_tokens_to_ids(p)].item() for p in pronouns}

def evaluate_bias_with_cohen(sentences, gold_pronouns, tokenizer, model):
    correct, total = 0, 0
    male_pref, female_pref = 0, 0

    assoc_scores = []

    for sent, gold in zip(sentences, gold_pronouns):
        probs = get_pronoun_probs(sent, tokenizer, model, ["he", "she", "him", "her"])
        if not probs:
            continue

        predicted = max(probs, key=probs.get)
        if predicted == gold:
            correct += 1


        male_score = probs["he"] + probs["him"]
        female_score = probs["she"] + probs["her"]
        assoc_scores.append(male_score - female_score)

        if predicted in ["he", "him"]:
            male_pref += 1
        else:
            female_pref += 1
        total += 1

    acc = correct / total if total > 0 else 0
    return acc, male_pref, female_pref, total, assoc_scores

def cohens_d(list1, list2):
    """Compute Cohen's d between two sets of scores."""
    x, y = np.array(list1), np.array(list2)
    nx, ny = len(x), len(y)
    mean_x, mean_y = x.mean(), y.mean()
    pooled_std = np.sqrt(((nx-1)*x.var(ddof=1) + (ny-1)*y.var(ddof=1)) / (nx+ny-2))
    return (mean_x - mean_y) / pooled_std


In [ ]:
pro_acc, pro_male, pro_female, pro_total, pro_scores = evaluate_bias_with_cohen(pro_sents, pro_labels, tokenizer, model)
anti_acc, anti_male, anti_female, anti_total, anti_scores = evaluate_bias_with_cohen(anti_sents, anti_labels, tokenizer, model)

# Cohen's d
d_score = cohens_d(pro_scores, anti_scores)

print("Pro accuracy:", pro_acc)
print("Anti accuracy:", anti_acc)
print("Gap:", pro_acc - anti_acc)
print("Cohen's d:", d_score)


Pro accuracy: 0.5489130434782609
Anti accuracy: 0.52
Gap: 0.028913043478260847
Cohen's d: 0.02907922650786027
